<a href="https://colab.research.google.com/github/harsh-hy/DeepLearning/blob/main/CIFAR_10_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, _), (_,_ ) = tf.keras.datasets.cifar10.load_data()

x_train = (x_train.astype("float32") - 127.5) / 127.5
x_train = x_train.reshape(x_train.shape[0] , 32,32,3)

In [ ]:
def build_generator():
  model= tf.keras.Sequential([
      layers.Dense(4*4*256, use_bias=False, input_shape=(100,)),
      layers.BatchNormalization(),
      layers.LeakyReLU(),

      layers.Reshape((4,4,256)),

      layers.Conv2DTranspose(128, (5,5), strides=(2,2), padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.LeakyReLU(),

      layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
      layers.BatchNormalization(),
      layers.LeakyReLU(),

      layers.Conv2DTranspose(3, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh')
  ])

  return model

In [ ]:
def build_discriminator():
  model= tf.keras.Sequential([
      layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[32,32,3]),
      layers.LeakyReLU(),
      layers.Dropout(0.3),

      layers.Conv2D(128, (5,5), strides=(2,2), padding='same'),
      layers.LeakyReLU(),
      layers.Dropout(0.3),

      layers.Flatten(),
      layers.Dense(1, activation='sigmoid')
      ])
  return model

In [ ]:
generator= build_generator()
discriminator= build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002,0.5), metrics=['accuracy'])

In [ ]:
discriminator.trainable=False
z=layers.Input(shape=(100,))
img=generator(z)
valid=discriminator(img)
gan=tf.keras.Model(z,valid)
gan.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(0.0002,0.5))

In [ ]:
def show_generated_images(generator, epoch, n=9):
  noise=np.random.normal(0,1,size=[n,100])
  generated_images=generator.predict(noise)
  generated_images=generated_images.reshape(n,32,32,3)

  plt.figure(figsize=(10,10))
  for i in range(n):
    plt.subplot(3,3,i+1)
    plt.imshow(generated_images[i])
    plt.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
def train_gan(epochs=5000, batch_size=128):
  half_batch=batch_size//2
  for epoch in range(epochs):
    idx=np.random.randint(0,x_train.shape[0],half_batch)
    real_imgs= x_train[idx]

    noise=np.random.normal(0,1,(half_batch,100))
    fake_imgs=generator.predict(noise)

    d_loss_real=discriminator.train_on_batch(real_imgs,np.ones((half_batch,1)))
    d_loss_fake=discriminator.train_on_batch(fake_imgs,np.zeros((half_batch,1)))
    d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

    noise= np.random.normal(0, 1, (batch_size, 100))
    valid_y= np.ones((batch_size, 1))
    g_loss= gan.train_on_batch(noise, valid_y)

    if epoch % 200 == 0:
      print(f'{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]')
      show_generated_images(generator,epoch)

In [ ]:
train_gan(epochs=5000, batch_size=128)